In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
from leaderboard_utils import GAME_ORDER


# Load the rank data
with open('rank_data_03_25_2025.json', 'r') as f:
    rank_data = json.load(f)

# Get combined leaderboard with all games
selected_games = {game: True for game in GAME_ORDER}
df = get_combined_leaderboard(rank_data, selected_games)

def create_group_bar_chart(df):
    """
    Create an interactive grouped bar chart comparing AI model performance across different games
    
    Args:
        df (pd.DataFrame): DataFrame containing the combined leaderboard data
        
    Returns:
        plotly.graph_objects.Figure: The generated interactive group bar chart figure
    """
    # Get active games (those that have score columns in the DataFrame)
    active_games = []
    for game in GAME_ORDER:
        score_col = f"{game} Score"
        if score_col in df.columns:
            active_games.append(game)
    
    if not active_games:
        return None  # Return None if no games are selected

    # Prepare data for plotting
    plot_data = []
    
    for game in active_games:
        score_col = f"{game} Score"
        for _, row in df.iterrows():
            model = row['Player']
            score = row[score_col]
            
            if score != '_' and float(score) > 0:  # Only include non-zero scores
                plot_data.append({
                    'Game': game,
                    'Model': model,
                    'Score': float(score)
                })
    
    # Create DataFrame for plotting
    plot_df = pd.DataFrame(plot_data)
    
    # Calculate normalized scores for each game
    normalized_scores = []
    for game in active_games:
        game_scores = plot_df[plot_df['Game'] == game]['Score']
        mean = game_scores.mean()
        std = game_scores.std()
        
        # Normalize scores using z-score and scale to 0-100 range
        if std == 0:
            normalized = [50 if s > 0 else 0 for s in game_scores]
        else:
            z_scores = [(s - mean) / std for s in game_scores]
            normalized = [max(0, min(100, (z * 30) + 50)) for z in z_scores]
        
        normalized_scores.extend(normalized)
    
    plot_df['Normalized Score'] = normalized_scores
    
    # Create the interactive bar chart
    fig = px.bar(plot_df,
                 x='Game',
                 y='Normalized Score',
                 color='Model',
                 title='AI Model Performance Across Games',
                 labels={
                     'Normalized Score': 'Normalized Performance Score',
                     'Game': 'Game',
                     'Model': 'AI Models'
                 },
                 template='plotly_white',
                 barmode='group',
                 height=600)
    
    # Customize the layout
    fig.update_layout(
        title_x=0.5,
        title_font_size=20,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        legend_title_font_size=14,
        legend_font_size=12,
        xaxis_tickangle=45,
        margin=dict(l=50, r=50, t=80, b=50),
        hovermode='x unified'
    )
    
    # Add hover template
    fig.update_traces(
        hovertemplate='<b>%{x}</b><br>' +
                     'Model: %{fullData.name}<br>' +
                     'Normalized Score: %{y:.1f}<br>' +
                     'Raw Score: %{customdata[0]:.1f}<extra></extra>',
        customdata=plot_df[['Score']].values
    )
    
    return fig

fig = create_group_bar_chart(df)
fig.show()

NameError: name 'df' is not defined